In [ ]:
from vlm_worker import VLMWorker
import torch
import time
import argparse
import numpy as np
from PIL import Image
from transformers import AutoProcessor, AutoModelForImageTextToText

# --- Constants ---
MODEL_ID = "Qwen/Qwen3-VL-2B-Instruct"
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480

class DataGenerator:
    """Generates synthetic turn data."""
    def __init__(self, width, height, processor):
        self.width = width
        self.height = height
        self.processor = processor

    def create_synthetic_image(self):
        arr = np.random.randint(0, 255, (self.height, self.width, 3), dtype=np.uint8)
        return Image.fromarray(arr)

    def _prepare_turn_inputs(self, step_idx):
        """
        Creates inputs for a SINGLE turn (Image + Text).
        We do not build the full conversation history in the prompt.
        We rely on the KV cache for history.
        """
        image = self.create_synthetic_image()
        
        # Construct a standalone prompt for this step
        # We simulate the user asking for a move
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": f"Step {step_idx}: Next move?"}
                ]
            },
            # We add the Assistant start token to force the model to predict the response immediately
            {"role": "assistant", "content": "**forward**"} 
        ]
        return messages, [image]

worker = VLMWorker()
generator = DataGenerator(IMAGE_WIDTH, IMAGE_HEIGHT, worker.processor)


Loading Qwen/Qwen3-VL-2B-Instruct...


In [2]:
worker.reset()
from tqdm import tqdm
for i in tqdm(range(100)):
    messages,images = generator._prepare_turn_inputs(i)
    action_probs = worker.infer_probs(messages,images)

  0%|          | 0/100 [00:10<?, ?it/s]


AssertionError: 